<a href="https://colab.research.google.com/github/ddesai-sjsu/Machine-Learning/blob/main/Homework_10_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk

In [ ]:
import pickle
import re
from nltk.tokenize import word_tokenize

In [ ]:
import nltk
from collections import Counter
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader')

from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
import scipy

In [ ]:
def get_POS_tags(statement):
  tokens = nltk.word_tokenize(statement)
  text = nltk.Text(tokens)
  tags = nltk.pos_tag(text)
  counts = Counter(tag for word,tag in tags)
  noun_counts=counts['NN']+counts['NNS']+counts['NNP']+counts['NNPS']
  verb_counts=counts['VB']+counts['VBD']+counts['VBG']+counts['VBN']+counts['VBP']+counts['VBG']
  adverbverb_counts=counts['VB']+counts['VBD']+counts['VBG']+counts['VBN']+counts['VBP']+counts['VBG']
  adjective_counts=counts['JJ']+counts['JJR']+counts['JJS']
  possessive_ending_counts=counts['POS']
  return noun_counts,verb_counts,adverbverb_counts,adjective_counts,possessive_ending_counts

In [ ]:
def get_sentiment_score(statement):
  model=pickle.load(open('/content/Neural Net', 'rb'))
  tfidf_vect=pickle.load(open('/content/tfidf', 'rb'))
  stop_words = set(stopwords.words('english'))
  stemmer = SnowballStemmer("english")
  lem=WordNetLemmatizer()
  analyzer = SentimentIntensityAnalyzer()

  p_statement= re.sub(r"\W", " ",str(statement))
  p_statement=re.sub(r'[,\.!?]', " ",str(p_statement))
  p_statement=re.sub(r" \d+", " ",str(p_statement))
  p_statement=word_tokenize(p_statement.lower())
  p_statement=' '.join(p_statement)
  p_statement=' '.join([word for word in p_statement.split() if word not in stop_words])
  p_statement=' '.join([lem.lemmatize(word) for word in p_statement.split()])
  p_statement=' '.join([stemmer.stem(word) for word in p_statement.split()])
  noun_counts,verb_counts,adverbverb_counts,adjective_counts,possessive_ending_counts=get_POS_tags(p_statement)

  compound= analyzer.polarity_scores(p_statement)['compound']
  neg=  analyzer.polarity_scores(p_statement)['neg'] 
  neu = analyzer.polarity_scores(p_statement)['neu']
  pos = analyzer.polarity_scores(p_statement)['pos']

  X=[noun_counts,verb_counts,adverbverb_counts,adjective_counts,possessive_ending_counts,compound,neg,neu,pos]
  vector=tfidf_vect.transform([p_statement])

  final_input=scipy.sparse.hstack((vector,X),format='csr')
  
  prediction=model.predict(final_input)
  prediction_probability=model.predict_proba(final_input)[:,1]
  return prediction_probability.item(0)

In [ ]:
get_sentiment_score('sad')

In [1]:
def isFakeNews(statement,): #add your parameters here
  accur = [0.2257,] #add your model accuracy here
  w = [float(i)/sum(accur) for i in accur]
  sumW = 0
  prob = []
  if statement!='':
    prob.append(w[0] * get_sentiment_score(statement))
    sumW += w[0]
  ##do similar steps to your model
 
  probTotal = sum(prob[0:len(prob)]) / sumW
  return probTotal
    

SyntaxError: ignored

In [ ]:
# result=isFakeNews(pass parameters here)

In [ ]:
if result <=0.16:
    print('1')
elif result >0.16  and result <=0.32:
    print('2')
elif result >0.32  and result <=0.48:
    print('3')
elif result >0.48  and result <=0.64:
    print('4')
elif result >0.64  and result <=0.80:
    print('5')
elif result >0.80:
    print('6')